In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Recommended for the T4 GPU
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset

# Load the entire dataset
dataset = load_dataset("i-am-mushfiq/FirstAidQA", split="train")

# Peek at a few samples
print(dataset[0])
print(len(dataset))

{'question': 'When should you move an injured person at an accident site?', 'answer': "You should only move an injured person if there is immediate danger such as a fire, oncoming traffic, or toxic fumes. Otherwise, it's best to leave them where they are, administer first aid on the spot, and wait for professional medical help to arrive."}
5550


In [ ]:
from unsloth.chat_templates import get_chat_template


# 2️⃣ Get the tokenizer with Unsloth’s chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# 3️⃣ Reformat each example into a chat-style conversation
#    compatible with the Llama 3.1 instruction format
def formatting_prompts_func(examples):
    # Convert each question–answer pair into a chat conversation
    convos = []
    for q, a in zip(examples["question"], examples["answer"]):
        # append some context to each convo
        convos.append([
            {"role": "user", "content": f"You are a certified first aid assistant. {q}"},
            {"role": "assistant", "content": a},
        ])

    # Apply the chat template
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        for convo in convos
    ]

    return {"text": texts}

# 4️⃣ Map the function over the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Optional sanity check
print(dataset[5])

Map:   0%|          | 0/5550 [00:00<?, ? examples/s]

{'question': "In what scenario is a 'human crutch' technique most suitable?", 'answer': 'The human crutch technique is most suitable for an injured adult who can still move with some assistance, such as someone with a sprained ankle or mild leg injury. It allows the injured person to walk with the support of another person.', 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are a certified first aid assistant. In what scenario is a 'human crutch' technique most suitable?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe human crutch technique is most suitable for an injured adult who can still move with some assistance, such as someone with a sprained ankle or mild leg injury. It allows the injured person to walk with the support of another person.<|eot_id|>"}


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Training for 3 epochs
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 10,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        num_train_epochs = 3, # Set this for 1 full training run.
        # TODO: test run to see how fast it goes
        # max_steps = 320,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/llama_first_aid_finetune_3_epochs",
        report_to = "none",

        save_strategy = "steps",
        save_steps = 30,             # checkpoint every 100 steps
        save_total_limit = 20,         # keep last 3 checkpoints
        ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5550 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only

# compute loss only on system responses
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=6):   0%|          | 0/5550 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are a certified first aid assistant. In what scenario is a 'human crutch' technique most suitable?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe human crutch technique is most suitable for an injured adult who can still move with some assistance, such as someone with a sprained ankle or mild leg injury. It allows the injured person to walk with the support of another person.<|eot_id|>"

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                          The human crutch technique is most suitable for an injured adult who can still move with some assistance, such as someone with a sprained ankle or mild leg injury. It allows the injured person to walk with the support of another person.<|eot_id|>'

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.047 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,550 | Num Epochs = 3 | Total steps = 417
O^O/ \_/ \    Batch size per device = 10 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (10 x 4 x 1) = 40
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss
1,1.340500
2,1.302100
3,1.381900
4,1.496600
5,1.461600
6,1.407500
7,1.321900
8,1.401000
9,1.346500
10,1.422300


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

998.8234 seconds used for training.
16.65 minutes used for training.
Peak reserved memory = 4.047 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 27.454 %.
Peak reserved memory for training % of max memory = 0.0 %.
